# ORTOOLS  알고리즘 구현

In [1]:
import pandas as pd
pd.read_csv("../data/final_num_171.csv")
distance_matrix=pd.read_csv("../data/final_distance.csv")

In [3]:

matrix = distance_matrix.values
matrix[[0, 171]] = matrix[[171, 0]]

# 열 교체
matrix[:, [0, 171]] = matrix[:, [171, 0]]
lst=matrix
demands=[0,612, 306, 921, 982, 231, 1068, 173, 151, 35, 612, 122, 122, 140, 414, 45, 182, 224, 122, 306, 45, 45, 136, 224, 133, 60, 140, 61, 538, 136, 758, 224, 196, 61, 45, 820, 97, 35, 61, 285, 97, 224, 232, 232, 98, 98, 52, 605, 97, 133, 140, 140, 140, 97, 136, 97, 136, 136, 97, 133, 98, 61, 215, 125, 136, 136, 94, 94, 136, 280, 94, 97, 173, 1465, 35, 285, 384, 217, 332, 258, 984, 112, 73, 73, 151, 332, 1043, 440, 440, 196, 258, 760, 136, 332, 112, 378, 196, 52, 112, 1010, 151, 71, 950, 160, 378, 432, 280, 605, 210, 35, 35, 35, 125, 217, 173, 71, 322, 322, 436, 479, 1065, 387, 173, 322, 322, 71, 564, 423, 432, 215, 215, 92, 52, 160, 52, 92, 92, 160, 160, 280, 397, 35, 35, 35, 35, 35, 35, 35, 35, 35, 1231, 61, 61, 60, 35, 35, 35, 35, 35, 35, 60, 61, 94, 35, 60, 61, 182, 60, 136, 136, 414, 94]

In [20]:
print(sum(demands)-sum([6100, 5000,3700 , 3000,1500])*2)


1837


In [5]:
"""Capacited Vehicles Routing Problem (CVRP)."""
import numpy as np 
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = lst
    data["demands"] = demands
    data["vehicle_capacities"] = [6100, 5000,3700 , 3000,1500]*2+[3000]
    data["num_vehicles"] = 11
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    #print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    routelst=[]

    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
    #    print(plan_output)
        routelst.append(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    #print(f"Total load of all routes: {total_load}")
    return solution.ObjectiveValue(),total_distance,total_load,routelst

def main(first,local_search):
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        first
        #routing_enums_pb2.FirstSolutionStrategy.SAVINGS
    )
    search_parameters.local_search_metaheuristic = (
        local_search
        #routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(30)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        return (print_solution(data, manager, routing, solution))
    else:
        return "no result","no result","no result","no result"
result=[]

# FirstSolutionStrategy와 LocalSearchMetaheuristic의 문자열과 실제 열거형 값을 매핑하는 딕셔너리 생성
first_solution_strategy_mapping = {
    'GLOBAL_CHEAPEST_ARC': routing_enums_pb2.FirstSolutionStrategy.GLOBAL_CHEAPEST_ARC,
    'LOCAL_CHEAPEST_ARC': routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_ARC,
    'PATH_CHEAPEST_ARC': routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC,
    'PATH_MOST_CONSTRAINED_ARC': routing_enums_pb2.FirstSolutionStrategy.PATH_MOST_CONSTRAINED_ARC,
    'EVALUATOR_STRATEGY': routing_enums_pb2.FirstSolutionStrategy.EVALUATOR_STRATEGY,
    'ALL_UNPERFORMED': routing_enums_pb2.FirstSolutionStrategy.ALL_UNPERFORMED,
    'BEST_INSERTION': routing_enums_pb2.FirstSolutionStrategy.BEST_INSERTION,
    'PARALLEL_CHEAPEST_INSERTION': routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION,
    'LOCAL_CHEAPEST_INSERTION': routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_INSERTION,
    'SAVINGS': routing_enums_pb2.FirstSolutionStrategy.SAVINGS,
    'SWEEP': routing_enums_pb2.FirstSolutionStrategy.SWEEP,
    'FIRST_UNBOUND_MIN_VALUE': routing_enums_pb2.FirstSolutionStrategy.FIRST_UNBOUND_MIN_VALUE,
    'CHRISTOFIDES': routing_enums_pb2.FirstSolutionStrategy.CHRISTOFIDES,
}

local_search_metaheuristic_mapping = {
    'GREEDY_DESCENT': routing_enums_pb2.LocalSearchMetaheuristic.GREEDY_DESCENT,
    'GUIDED_LOCAL_SEARCH': routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,
    'SIMULATED_ANNEALING': routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING,
    'TABU_SEARCH': routing_enums_pb2.LocalSearchMetaheuristic.TABU_SEARCH,
    'GENERIC_TABU_SEARCH': routing_enums_pb2.LocalSearchMetaheuristic.GENERIC_TABU_SEARCH,
}

# 문자열 리스트를 실제 열거형 값으로 변환
first_solution_strategies_ordered = [
    'GLOBAL_CHEAPEST_ARC',
    'LOCAL_CHEAPEST_ARC',
    'PATH_CHEAPEST_ARC',
    'PATH_MOST_CONSTRAINED_ARC',
    'EVALUATOR_STRATEGY',
    'ALL_UNPERFORMED',
    'BEST_INSERTION',
    'PARALLEL_CHEAPEST_INSERTION',
    'LOCAL_CHEAPEST_INSERTION',
    'SAVINGS',
    'SWEEP',
    'FIRST_UNBOUND_MIN_VALUE',
    'CHRISTOFIDES'
]

local_search_metaheuristics_ordered = [
    'GREEDY_DESCENT',
    'GUIDED_LOCAL_SEARCH',
    'SIMULATED_ANNEALING',
    'TABU_SEARCH',
    'GENERIC_TABU_SEARCH'
]
matrix_obj=np.zeros((len(first_solution_strategies_ordered),len(local_search_metaheuristics_ordered)))
matrix_dis=np.zeros((len(first_solution_strategies_ordered),len(local_search_metaheuristics_ordered)))
matrix_load=np.zeros((len(first_solution_strategies_ordered),len(local_search_metaheuristics_ordered)))
# 반복문에서 main 함수 실행
final_route=[]
for strategy in range(len(first_solution_strategies_ordered)):
    for heuristic in range(len(local_search_metaheuristics_ordered)):
        first_solution_strategy = first_solution_strategy_mapping[first_solution_strategies_ordered[strategy]]
        local_search_metaheuristic = local_search_metaheuristic_mapping[local_search_metaheuristics_ordered[heuristic]]
        print(f"{strategy} {heuristic} :", end="")
        obj,dis,load,route=main(first_solution_strategy, local_search_metaheuristic)
        if obj=="no result":
            pass
        else:
            matrix_obj[strategy][heuristic]=obj
            matrix_dis[strategy][heuristic]=dis
            matrix_load[strategy][heuristic]=load
            final_route.append(route)
        

0 0 :0 1 :0 2 :0 3 :0 4 :1 0 :1 1 :1 2 :1 3 :1 4 :2 0 :Total distance of all routes: 133749m
2 1 :Total distance of all routes: 133267m
2 2 :Total distance of all routes: 133670m
2 3 :Total distance of all routes: 132446m
2 4 :Total distance of all routes: 133749m
3 0 :Total distance of all routes: 133749m
3 1 :Total distance of all routes: 133267m
3 2 :Total distance of all routes: 133670m
3 3 :Total distance of all routes: 132446m
3 4 :Total distance of all routes: 133749m
4 0 :4 1 :4 2 :4 3 :4 4 :5 0 :5 1 :5 2 :5 3 :5 4 :6 0 :6 1 :6 2 :6 3 :6 4 :7 0 :Total distance of all routes: 136576m
7 1 :Total distance of all routes: 134934m
7 2 :Total distance of all routes: 135030m
7 3 :Total distance of all routes: 136576m
7 4 :Total distance of all routes: 136576m
8 0 :Total distance of all routes: 137383m
8 1 :Total distance of all routes: 133519m
8 2 :Total distance of all routes: 137590m
8 3 :Total distance of all routes: 135763m
8 4 :Total distance of all routes: 136190m
9 0 :Total dist

# 알고리즘 구현 결과 

## 1. 목적함수(거리) 결과

In [63]:
df=pd.DataFrame(matrix_dis,index=first_solution_strategies_ordered,columns=local_search_metaheuristics_ordered)
df = df[(df != 0.0).any(axis=1)]
df

,GREEDY_DESCENT,GUIDED_LOCAL_SEARCH,SIMULATED_ANNEALING,TABU_SEARCH,GENERIC_TABU_SEARCH
PATH_CHEAPEST_ARC,133749.0,133267.0,133670.0,132446.0,133749.0
PATH_MOST_CONSTRAINED_ARC,133749.0,133267.0,133670.0,132446.0,133749.0
PARALLEL_CHEAPEST_INSERTION,136576.0,134934.0,135030.0,136576.0,136576.0
LOCAL_CHEAPEST_INSERTION,137383.0,133519.0,137590.0,135763.0,136190.0
SAVINGS,137976.0,131097.0,137976.0,137220.0,137976.0
CHRISTOFIDES,136062.0,134726.0,136062.0,136062.0,136062.0


In [69]:
import re
def jingyu(final_route):
    distances = []
    loads = []
    for route in final_route:
        distance_match = re.search(r'Distance of the route: (\d+)m', route)
        load_match = re.search(r'Load of the route: (\d+)', route)
        if distance_match and load_match:
            distances.append(int(distance_match.group(1)))
            loads.append(int(load_match.group(1)))

    # 결과 출력
    distance_A,distance_B,distance_C,distance_D,distance_E=0,0,0,0,0
    # for i, (distance, load) in enumerate(zip(distances, loads)):
    #     print(f"Vehicle {i}: Distance = {distance}m, Load = {load}")
    for i in range(10):
        if i%5==0:
            distance_A+=distances[i]
        elif i%5==1:
            distance_B+=distances[i]
        elif i%5==2:
            distance_C+=distances[i]
        elif i%5==3:
            distance_D+=distances[i]
        else:
            distance_E+=distances[i]
    distance_D+=distances[-1]
    return np.array([distance_A,distance_B,distance_C,distance_D,distance_E])


## 2. 알고리즘별 차량별 이동량

In [4]:
import pandas as pd

# 주어진 데이터
data = {
    '차량1': [25853, 25853, 25605, 25452, 25853, 25853, 25853, 25605, 25452, 25853, 30798, 30029, 30771, 30798, 30798, 30597, 33554, 36159, 34532, 35093, 31704, 26229, 31704, 31799, 31704, 25690, 27041, 25690, 25690, 25690],
    '차량2': [25893, 25810, 25893, 26049, 25893, 25893, 25810, 25893, 26049, 25893, 27130, 27593, 27681, 27130, 27130, 27157, 23935, 25802, 25433, 25433, 26136, 23595, 26136, 25890, 26136, 28346, 31983, 28346, 28346, 28346],
    '차량3': [24128, 23305, 24297, 24208, 24128, 24128, 23305, 24297, 24208, 24128, 24823, 24833, 23577, 24823, 24823, 25260, 23496, 23203, 23777, 23238, 27615, 28056, 27615, 27311, 27615, 22132, 21950, 22132, 22132, 22132],
    '차량4': [38443, 38443, 38443, 38244, 38443, 38443, 38443, 38443, 38244, 38443, 33063, 32097, 33334, 33063, 33063, 35010, 34939, 33067, 32662, 33067, 34437, 34969, 34437, 33881, 34437, 39621, 33479, 39621, 39621, 39621],
    '차량5': [19432, 19856, 19432, 18493, 19432, 19432, 19856, 19432, 18493, 19432, 20762, 20382, 19667, 20762, 20762, 19359, 17595, 19359, 19359, 19359, 18084, 18248, 18084, 18339, 18084, 20273, 20273, 20273, 20273, 20273]
}

# DataFrame 생성
df = pd.DataFrame(data)

# 결과 출력
df

,차량1,차량2,차량3,차량4,차량5
0,25853,25893,24128,38443,19432
1,25853,25810,23305,38443,19856
2,25605,25893,24297,38443,19432
3,25452,26049,24208,38244,18493
4,25853,25893,24128,38443,19432
5,25853,25893,24128,38443,19432
6,25853,25810,23305,38443,19856
7,25605,25893,24297,38443,19432
8,25452,26049,24208,38244,18493
9,25853,25893,24128,38443,19432


In [71]:
lst=[]
for i in final_route:
          lst.append(jingyu(i))
pd.DataFrame(lst,columns=["차량1","차량2","차량3","차량4","차량5"])

,차량1,차량2,차량3,차량4,차량5
0,25853,25893,24128,38443,19432
1,25853,25810,23305,38443,19856
2,25605,25893,24297,38443,19432
3,25452,26049,24208,38244,18493
4,25853,25893,24128,38443,19432
5,25853,25893,24128,38443,19432
6,25853,25810,23305,38443,19856
7,25605,25893,24297,38443,19432
8,25452,26049,24208,38244,18493
9,25853,25893,24128,38443,19432


In [7]:
pd.set_option('display.float_format', '{:.1f}'.format)

oil_1=1490/5600
oil_2=1490/8200
oil_3=1490/6700
oil_4=1490/10600
oil_5=1490/8400
cost_1=(105963636+4287600)/8/312
cost_2=(84916364 +3857660)/8/312
cost_3=(92380000 + 3153620)/8/312
cost_4=(64863636 + 3119110)/8/312
cost_5=(102835455 + 3041100)/8/312
lst=[]
df.assign(cost=lambda x:x.차량1*oil_1+x.차량2*oil_2+x.차량3*oil_3+x.차량4*oil_4+x.차량5*oil_5+cost_1+cost_2+cost_3+cost_5)

,차량1,차량2,차량3,차량4,차량5,cost
0,25853,25893,24128,38443,19432,186231.0
1,25853,25810,23305,38443,19856,186108.1
2,25605,25893,24297,38443,19432,186202.6
3,25452,26049,24208,38244,18493,185975.9
4,25853,25893,24128,38443,19432,186231.0
5,25853,25893,24128,38443,19432,186231.0
6,25853,25810,23305,38443,19856,186108.1
7,25605,25893,24297,38443,19432,186202.6
8,25452,26049,24208,38244,18493,185975.9
9,25853,25893,24128,38443,19432,186231.0


In [9]:
import numpy as np

## 3.알고리즘별 표준편차

In [11]:
pd.set_option('display.float_format', '{:.1f}'.format)
m=np.zeros([len(df.index),len(df.columns)])
for i in range(len(df.index)):
          for j in range(len(df.columns)):
                m[i][j]=np.std(jingyu(final_route[i*5+j]))
pd.DataFrame(m,columns=df.columns,index=df.index)

NameError: name 'jingyu' is not defined